In [1]:
import mysql.connector
from mysql.connector import errorcode

In [11]:
try:
    cnx = mysql.connector.connect(user='u179344db1', password='0.q71kn469pp',
                                  host=' e115240-mysql.services.easyname.eu',
                                  database='u179344db1')
    print("connection successfully established.")
    
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Invalid user credentials.")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
    cnx.close()

connection successfully established.


In [12]:
cnx.close()